### Compute Target, Path, LSAP factor
- follow methodology of Swanson (2021)

In [469]:
import numpy as np
import pandas as pd
import os
from scipy.optimize import minimize
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import root
from functools import partial
import statsmodels.formula.api as smf

In [470]:
def objective1(u13_u23, F_pre):
    u13, u23 = u13_u23  # Unpack the vector
    u = np.array([[u13, u23, 1]])
    return u @ F_pre.T @ F_pre @ u.T

def constraint1(lambda1, u13_u23):
    u13, u23 = u13_u23  # Unpack the vector
    u = np.array([u13, u23, 1]) #Make u a vector rather than a matrix
    return np.dot(lambda1, u)

def objective2(u12_u22,lambda1, U3):
    u12, u22 = u12_u22
    left = np.vstack([lambda1,U3])
    right = np.array([u12, u22, 1])
    return left @ right.T

def objective3(u11_u21, U2, U3):
    u11, u21 = u11_u21
    left = np.vstack([U2.T, U3.T])
    right = np.array([u11, u21, 1])
    return left @ right

In [471]:
# input: T x n matrix of n asset price responses on T dates
def swanson_compute(surprises, len_pre, securities):
    X = surprises.drop('Date', axis = 1)
    scaler = StandardScaler()
    normalized_X = scaler.fit_transform(X)

    pca = PCA(n_components=3)
    pca.fit(normalized_X)
    factors = pca.transform(normalized_X)  # T x k  Principal component scores
    loadings = pca.components_
    #new
    F_pre = factors[:len_pre]
    lambda1 = loadings[:,0]

    # First equation to solve
    initial_guess = [0, 0]
    cons = {'type': 'eq', 'fun': lambda x: constraint1(lambda1, x)}
    result = minimize(lambda x: objective1(x, F_pre), initial_guess, method='SLSQP', constraints=cons)
    
    u13, u23 = result.x
    U3_sol = np.array([u13, u23, 1])
    U3 = U3_sol / np.linalg.norm(U3_sol)

    #Second equation to solve
    equation2 = partial(objective2, lambda1 = lambda1, U3 = U3)
    initial_guess = [1, 1]
    solution2 = root(equation2, initial_guess)
    
    u12, u22 = solution2.x
    U2_sol = np.array([u12, u22, 1])
    U2 = U2_sol / np.linalg.norm(U2_sol)

    #Third equation to solve
    equation3 = partial(objective3, U2 = U2, U3 = U3)
    initial_guess = [1, 1]
    solution3 = root(equation3, initial_guess)

    u11, u21 = solution3.x
    U1_sol = np.array([u11, u21, 1])
    U1 = U1_sol / np.linalg.norm(U1_sol)

    #combine
    U = np.column_stack([U1, U2, U3])
    F_star = factors @ U
    F_star = pd.DataFrame(F_star, columns = ['Target', 'Path', 'QE'])

    merged_df = pd.concat([surprises, F_star], axis = 1)

    factors = ['Target', 'Path', 'QE']

    for i in range(3):
        product = merged_df[factors[i]] * merged_df[securities[i]]
        count = np.sum(product > 0)
        if factors[i] == 'QE':
            if count > (len(product) / 2):
                merged_df['QE'] *= -1
            break
        if count < (len(product) / 2):
            merged_df[factors[i]] *= -1

    ## Confirm solution satisfies requirements
    if lambda1.T @ U[:,1] > 0.0000001:
        print("Property 1 doesn't hold")

    if lambda1.T @ U[:,2] > 0.0000001:
        print("Property 2 doesn't hold")
    
    # Compare pre and post crisis variance of QE factor
    pre_crisis_var = merged_df.loc[:len_pre, "QE"].var()
    post_crisis_var = merged_df.loc[len_pre:, "QE"].var()

    print("Pre-Crisis Variance:", pre_crisis_var)
    print("Post-Crisis Variance:", post_crisis_var)

    print("Pre-Crisis Length:", len_pre)
    print("Post-Crisis Length: ", len(merged_df) - len_pre)

    return merged_df



In [472]:
# Read FOMC Surprise Data
folder_name = 'MP Surprises'
bauer_swanson_path = os.path.join(os.getcwd(), folder_name, 'monetary-policy-surprises-data.xlsx')

# use same columns as Braun et al. (2024) - "Measuring Monetary Policy in the UK"
bs_cols = ['Date', 'FF1', 'FF2', 'ED1', 'ED2', 'ED4', 'TNOTE02', 'TNOTE05','TNOTE10']

bauer_swanson_surprises = pd.read_excel(bauer_swanson_path, sheet_name = 'FOMC (update 2023)', usecols = bs_cols)

fomc_surprises = bauer_swanson_surprises[['Date', 'ED1', 'ED2', 'ED4', 'TNOTE02', 'TNOTE05', 'TNOTE10']]
fomc_surprises.rename(columns={'ED1': 'Q1', 'ED2': 'Q2', 'ED4': 'Y1', 'TNOTE02': 'Y2', 'TNOTE05': 'Y5', 'TNOTE10': 'Y10'}, inplace=True)
fomc_surprises = fomc_surprises.dropna()
fomc_surprises.reset_index(drop=True, inplace=True)

/var/folders/s6/xgx8wq8n63q742dx4hm1b4l00000gn/T/ipykernel_9573/3158798667.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fomc_surprises.rename(columns={'ED1': 'Q1', 'ED2': 'Q2', 'ED4': 'Y1', 'TNOTE02': 'Y2', 'TNOTE05': 'Y5', 'TNOTE10': 'Y10'}, inplace=True)


In [473]:
fomc_surprises

,Date,Q1,Q2,Y1,Y2,Y5,Y10
0,1991-01-08,-0.12500,-0.1100,-0.0800,-0.024492,-0.053286,-0.058557
1,1991-02-01,-0.04000,-0.0550,-0.0450,-0.016261,0.003629,0.006617
2,1991-02-01,-0.06000,-0.0300,-0.0300,-0.004875,-0.021771,-0.008824
3,1991-02-07,0.01000,-0.0050,-0.0100,-0.003248,0.000000,0.002192
4,1991-03-08,-0.06500,-0.0850,-0.0500,0.019617,-0.027666,-0.029307
...,...,...,...,...,...,...,...
291,2023-06-14,0.00250,0.0750,0.1450,0.115748,0.091064,0.054933
292,2023-07-26,-0.00250,0.0050,0.0075,0.004019,0.000000,0.000000
293,2023-09-20,0.03250,0.0625,0.1075,0.071429,0.057770,0.036750
294,2023-11-01,-0.01500,-0.0150,-0.0125,-0.015098,-0.014365,-0.008503


In [474]:
### Read surprise data for BoE
folder_name = 'MP Surprises'
boe_path = os.path.join(os.getcwd(), folder_name, 'UK_MPS.xlsx')

# use same columns as Braun et al. (2024) - "Measuring Monetary Policy in the UK"
boe_cols = ['Datetime', 'isMPC', 'FSScm1','FSScm2','FSScm3','FSScm4','SON3c1', 'SON3c2', 'SON3c3', 
            'SON3c4', 'GB2YT=RR', 'GB5YT=RR', 'GB10YT=RR', 'GBP1MOIS=']

boe_surprises = pd.read_excel(boe_path, sheet_name = 'Surprises', usecols = boe_cols)
boe_surprises

,Datetime,isMPC,FSScm1,FSScm2,FSScm3,FSScm4,GB10YT=RR,GB2YT=RR,GB5YT=RR,GBP1MOIS=,SON3c1,SON3c2,SON3c3,SON3c4
0,1997-06-06 11:00:00,True,0.07,0.02,0.020,0.025,0.0210,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN
1,1997-07-10 11:00:00,True,0.01,0.01,0.010,0.005,0.0000,0.0460,0.0080,NaN,NaN,NaN,NaN,NaN
2,1997-08-07 11:00:00,True,-0.07,-0.11,-0.100,-0.110,-0.0805,-0.0815,-0.0695,NaN,NaN,NaN,NaN,NaN
3,1997-08-13 09:30:00,False,-0.02,-0.02,-0.020,-0.020,-0.0085,-0.0175,-0.0245,NaN,NaN,NaN,NaN,NaN
4,1997-09-11 11:00:00,True,0.00,0.02,0.015,0.010,0.0000,0.0045,-0.0025,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,2024-02-01 12:30:00,False,NaN,NaN,NaN,NaN,-0.0480,-0.0425,-0.0510,-0.0016,0.000,-0.015,-0.0250,-0.035
394,2024-03-21 12:00:00,True,NaN,NaN,NaN,NaN,-0.0100,-0.0340,-0.0220,0.0000,0.000,-0.030,-0.0325,-0.030
395,2024-05-09 11:00:00,True,NaN,NaN,NaN,NaN,-0.0150,-0.0325,-0.0230,0.0116,0.005,-0.015,-0.0325,-0.035
396,2024-05-09 11:30:00,False,NaN,NaN,NaN,NaN,-0.0080,-0.0215,-0.0150,0.0000,0.000,-0.010,-0.0100,-0.015


In [475]:
# only take surprises associated with MPC decisions
boe_surprises = boe_surprises[boe_surprises['isMPC'] == 1]

boe_surprises = boe_surprises.copy()

# merge LIBOR and SONIA into one column
boe_surprises.loc[:, 'Q1'] = boe_surprises.loc[:, 'FSScm1'].combine_first(boe_surprises.loc[:, 'SON3c1'])
boe_surprises.loc[:, 'Q2'] = boe_surprises.loc[:, 'FSScm2'].combine_first(boe_surprises.loc[:, 'SON3c2'])
boe_surprises.loc[:, 'Y1'] = boe_surprises.loc[:, 'FSScm4'].combine_first(boe_surprises.loc[:, 'SON3c4'])

In [476]:
boe_surprises = boe_surprises.drop(columns=['FSScm1','FSScm2','FSScm3','FSScm4','SON3c1','SON3c2','SON3c3','SON3c4','isMPC', 'GBP1MOIS='])
boe_surprises = boe_surprises.rename(columns={'GB10YT=RR': 'Y10','GB5YT=RR': 'Y5','GB2YT=RR': 'Y2','Datetime':'Date'})

boe_surprises = boe_surprises.dropna()
boe_surprises.reset_index(drop=True, inplace=True)
boe_surprises = boe_surprises.reindex(columns=['Date', 'Q1', 'Q2', 'Y1', 'Y2', 'Y5', 'Y10'])
boe_surprises

,Date,Q1,Q2,Y1,Y2,Y5,Y10
0,1997-06-06 11:00:00,0.070,0.020,0.025,0.0000,0.0000,0.0210
1,1997-07-10 11:00:00,0.010,0.010,0.005,0.0460,0.0080,0.0000
2,1997-08-07 11:00:00,-0.070,-0.110,-0.110,-0.0815,-0.0695,-0.0805
3,1997-09-11 11:00:00,0.000,0.020,0.010,0.0045,-0.0025,0.0000
4,1997-10-09 11:00:00,0.030,0.040,0.040,-0.0045,-0.0075,-0.0040
...,...,...,...,...,...,...,...
283,2023-12-14 12:00:00,0.000,0.000,0.000,0.0475,0.0410,0.0420
284,2024-02-01 12:00:00,0.000,0.020,0.060,0.0485,0.0480,0.0330
285,2024-03-21 12:00:00,0.000,-0.030,-0.030,-0.0340,-0.0220,-0.0100
286,2024-05-09 11:00:00,0.005,-0.015,-0.035,-0.0325,-0.0230,-0.0150


In [477]:
# read ECB Surprise Data
folder_name = 'MP Surprises'
ecb_path = os.path.join(os.getcwd(), folder_name, 'Dataset_EA-MPD.xlsx')

ecb_cols = ['date', 'OIS_3M','OIS_6M','OIS_1Y','OIS_2Y','OIS_5Y','OIS_10Y','DE5Y','DE10Y']
ecb_surprises_full = pd.read_excel(ecb_path, sheet_name = 'Monetary Event Window', usecols = ecb_cols)

ecb_surprises_full = ecb_surprises_full.rename(columns={'date': 'Date'})
ecb_surprises_full['Date'] = pd.to_datetime(ecb_surprises_full['Date'], errors='coerce')

In [478]:
# use German sovereigns for 5Y and 10Y yields before August 2011, OIS afterwards
ecb_surprises_full['5Y' ] = ecb_surprises_full.apply(
    lambda row: row['DE5Y'] if row['Date'] < pd.to_datetime('2011-08-01') else row['OIS_5Y'], axis=1
)
ecb_surprises_full['10Y'] = ecb_surprises_full.apply(
    lambda row: row['DE10Y'] if row['Date'] < pd.to_datetime('2011-08-01') else row['OIS_10Y'], axis=1
)

ecb_surprises_full.drop(['OIS_5Y', 'OIS_10Y', 'DE5Y','DE10Y'], axis=1, inplace=True)

In [479]:
ecb_surprises = ecb_surprises_full[['Date', 'OIS_3M', 'OIS_6M', 'OIS_1Y', 'OIS_2Y', '5Y', '10Y']]
ecb_surprises = ecb_surprises.rename(columns={'OIS_3M': 'Q1', 'OIS_6M': 'Q2', 'OIS_1Y': 'Y1', 'OIS_2Y': 'Y2', '5Y': 'Y5', '10Y': 'Y10'})
ecb_surprises = ecb_surprises.dropna()
ecb_surprises.reset_index(drop=True, inplace=True)
ecb_surprises

,Date,Q1,Q2,Y1,Y2,Y5,Y10
0,1999-12-02 00:00:00,0.000000,0.000000,0.000000,0.400000,0.300000,-0.350000
1,1999-12-15 00:00:00,0.000000,0.000000,-0.100000,-0.200000,-0.600000,-0.250000
2,2000-01-05 00:00:00,-1.000000,-1.000000,-3.500000,-2.500000,-4.300000,-2.250000
3,2000-01-20 00:00:00,0.000000,0.000000,0.000000,1.000000,1.200000,1.000000
4,2000-02-03 00:00:00,-1.000000,0.000000,9.500000,1.000000,-0.500000,-4.500000
...,...,...,...,...,...,...,...
272,2023-05-04 00:00:00,-5.599999,-6.369996,-6.700015,-6.850004,-5.949998,-1.999998
273,2023-06-15 00:00:00,0.950003,1.990008,1.374984,-0.830007,-2.349997,-4.499984
274,2023-07-27 12:15:00,-1.134992,-2.095008,-3.600001,-4.700017,-4.699993,-3.500009
275,2023-09-14 12:15:00,6.220007,4.885006,2.745008,-0.300002,-2.399993,-2.799988


In [480]:
fomc_combined = swanson_compute(fomc_surprises, len(fomc_surprises[fomc_surprises['Date'] > pd.to_datetime('2008-11-01')]), ['Q1', 'Y1', 'Y10'])

Pre-Crisis Variance: 0.2963205219454853
Post-Crisis Variance: 0.46180312653719613
Pre-Crisis Length: 127
Post-Crisis Length:  169


In [481]:
fomc_combined['Target'].corr(fomc_combined['Path'])

np.float64(0.7232767993842121)

In [482]:
boe_combined = swanson_compute(boe_surprises, len(boe_surprises[boe_surprises['Date'] > pd.to_datetime('2009-03-01')]), ['Q1', 'Y1', 'Y10'])

Pre-Crisis Variance: 0.6893092489368985
Post-Crisis Variance: 0.6253863650582083
Pre-Crisis Length: 150
Post-Crisis Length:  138


In [483]:
boe_combined['Target'].corr(boe_combined['Path'])

np.float64(0.6986048289906582)

In [484]:
ecb_combined = swanson_compute(ecb_surprises, len(ecb_surprises[ecb_surprises['Date'] > pd.to_datetime('2015-01-01')]), ['Q1', 'Y1', 'Y10'])

Pre-Crisis Variance: 0.5268254534143577
Post-Crisis Variance: 0.6184751421210503
Pre-Crisis Length: 71
Post-Crisis Length:  206


### Normalize Coefficients

In [485]:
model = smf.ols('Q1 ~ Target + Path + QE', data = fomc_combined).fit()
coef = model.params['Target']
fomc_combined['Target'] = fomc_combined['Target'] * coef

In [486]:
model = smf.ols('Y1 ~ Target + Path + QE', data = fomc_combined).fit()
coef = model.params['Path']
fomc_combined['Path'] = fomc_combined['Path'] * coef

In [487]:
model = smf.ols('Y10 ~ Target + Path + QE', data = fomc_combined).fit()
coef = model.params['QE']
fomc_combined['QE'] = fomc_combined['QE'] * -coef

In [488]:
model = smf.ols('Q1 ~ Target + Path + QE', data = ecb_combined).fit()
coef = model.params['Target']
ecb_combined['Target'] = ecb_combined['Target'] * coef

In [489]:
model = smf.ols('Y1 ~ Target + Path + QE', data = ecb_combined).fit()
coef = model.params['Path']
ecb_combined['Path'] = ecb_combined['Path'] * coef

In [490]:
model = smf.ols('Y10 ~ Target + Path + QE', data = ecb_combined).fit()
coef = model.params['QE']
ecb_combined['QE'] = ecb_combined['QE'] * -coef

In [491]:
model = smf.ols('Q1 ~ Target + Path + QE', data = boe_combined).fit()
coef = model.params['Target']
boe_combined['Target'] = boe_combined['Target'] * coef

In [492]:
model = smf.ols('Y1 ~ Target + Path + QE', data = boe_combined).fit()
coef = model.params['Path']
boe_combined['Path'] = boe_combined['Path'] * coef

In [493]:
model = smf.ols('Y10 ~ Target + Path + QE', data = boe_combined).fit()
coef = model.params['QE']
boe_combined['QE'] = boe_combined['QE'] * -coef

In [494]:
names = ["fomc", "boe", "ecb"]

dataframes = [fomc_combined, boe_combined, ecb_combined]

with pd.ExcelWriter('factors.xlsx', engine='openpyxl') as writer:
    for i, frame in enumerate(dataframes):
        frame.to_excel(writer, sheet_name=names[i], index=False)